Point by point stage movement using set position


In [1]:
from pipython2 import GCSDevice
from pipython2 import gcscommands
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

In [2]:
#create device object
gcs = GCSDevice('E-727')

In [3]:
print(gcs)

GCSCommands(msgs=GCSMessages(interface=GCSDll(C:\ProgramData\PI\GCSTranslator\PI_GCS2_DLL_x64.dll)), id=-1)


In [4]:
#connect Piezo stage (check COM number and baudrate from PI umove app)
gcs.ConnectRS232(3,460800)


In [5]:
#check connection status and set servo mode ON
#Correct value like  "(c)2015 Physik Instrumente (PI) GmbH & Co. KG, E-727, 0116008043, 13.21.00.08"
print (gcs.qIDN())
gcs.SVO1([1,2,3],[1,1,1])
print(gcs.qSVO1([1,2,3]))  #1 - ON, 0 - OFF. For this script all values should be ON

(c)2015 Physik Instrumente (PI) GmbH & Co. KG, E-727, 0116008043, 13.21.00.08

1=1 
2=1 
3=1



In [17]:
#used basic functions defined here: C:\Anaconda3\Lib\site-packages\PIPython-1.3.9.40-py3.7.egg\pipython\gcscommands.py

def point_by_point_snake(gcs,x_points,y_points,dwell_time,
                       delta_dist,offset,rep = 10 ,trigger_channel = 1):
    """
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger on selected channel send signal when stage is on target
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
        rep : number of cycles of the movement (int, 0 means infinity, )
        trigger_channel : output trigger channel (int, in [1,2,3])
    """
    start =time.time()
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    gcs.MOV1([1,2,3],offset)
    time.sleep(max(dwell_time-0.01,0))
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    t1=time.time()
    print(0,0,t1-start)
    tnow= time.time()
    for a in range(rep):
        try:
            for i in range(int(x_points/2)):
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])

                    gcs.CTO1(trigger_channel,[2,3],[1,2])
                    time.sleep(max(dwell_time-0.01,0))
                    print(2*i,j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1),offset[2]])
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                time.sleep(max(dwell_time-0.01,0))

                print((2*i+1),j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                tnow= time.time()
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])

                    gcs.CTO1(trigger_channel,[2,3],[1,2]) 

                    time.sleep(max(dwell_time-0.01,0))                    
                    print(2*i+1,y_points-j-2,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                gcs.CTO1(trigger_channel,[2,3],[1,2])                
                
                time.sleep(max(dwell_time-0.01,0))

                print(2*i+2,0,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                tnow= time.time()
        except KeyboardInterrupt:
            break
            
        
def move_to_point(gcs, offset, trigger_channel = 1):
    """
        Move stage command to selected position in 3D and trigger signal after arrival  
        gcs : piezo stage object

        offset : start position of the stage ([float,float,float], range[0,200])
        trigger_channel : output trigger channel (int, in [1,2,3])
    """    
    if max(offset)>200 or min(offset)<0:
        print('Set correct position between(0,200)')
    else:
            
        gcs.CTO1(trigger_channel,[2,3],[1,2])
        gcs.MOV1([1,2,3],offset)


In [7]:
point_by_point_snake_3(gcs,4,4,0.2,
                       0.1,[25.,25.,25.],rep=1,velocity=0.1)


1.4
0 0 0.008999824523925781


In [7]:
move_to_point(gcs, [180.,180.,180.])

NameError: name 'move_to_point' is not defined

In [10]:
gcs.CloseConnection()

In [10]:
vel =0.1*1e6/(2*50)
gcs.VEL1([1,2,3],[vel,vel,vel])

In [ ]:
def point_by_point_snake(gcs,x_points,y_points,dwell_time,
                       delta_dist,offset,rep = 10 ,trigger_channel = 1):
    """
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger on selected channel send signal when stage is on target
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
        rep : number of cycles of the movement (int, 0 means infinity, )
        trigger_channel : output trigger channel (int, in [1,2,3])
    """
    
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    gcs.MOV1([1,2,3],offset)
    time.sleep(max(dwell_time-0.01,0))
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    print(0,0)
    x_pos=np.arange(offset[0],x_points*delta_dist,delta_dist)
    y_pos=np.arange(offset[1],y_points*delta_dist,delta_dist)
    for a in range(rep):
        try:
            for i in range(int(x_points/2)):
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])
                    time.sleep(max(dwell_time-0.01,0))
                    gcs.CTO1(trigger_channel,[2,3],[1,2])
                    print(2*i,j+1)
                gcs.CTO1(trigger_channel,[2,3],[2,2])
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1),offset[2]])
                time.sleep(max(dwell_time-0.01,0))
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                print((2*i+1),j+1)
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    time.sleep(max(dwell_time-0.01,0))
                    gcs.CTO1(trigger_channel,[2,3],[1,2])    
                    print(2*i+1,y_points-j-2)
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                time.sleep(max(dwell_time-0.01,0))
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                print(2*i+2,0)
        except KeyboardInterrupt:
            break

##Correctting


In [6]:
def point_by_point_snake_2(gcs,x_points,y_points,dwell_time,
                       delta_dist,offset,rep = 10):
    """
        V1: Using trigger mode 0
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger signals reresent new point/line/frame(channels 1/2/3  respectively)
        
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
        rep : number of cycles of the movement (int, 0 means infinity, )
    """
    #Important parameters
    half = dwell_time *1/2
    #Setting velocity of the piezo (required to be low enough for trigger step size)
    vel =delta_dist*1e6/(25)
    gcs.VEL1([1,2,3],[vel,vel,vel])
    #Snake start 
    start =time.time()
    #Set in point nerby starting position(for trigger)
    gcs.MOV1([1,2,3],[offset[0]-delta_dist/4.,offset[1]-delta_dist/4.,offset[2]--delta_dist/4.])
    #Set triggers
    gcs.CTO1(1,[2,3,1],[1,0,delta_dist])
    gcs.CTO1(2,[2,3,1],[2,0,delta_dist])
    gcs.CTO1(3,[2,3,1],[3,0,delta_dist])

    gcs.MOV1([1,2,3],[offset[0]-delta_dist/2.,offset[1]-delta_dist/2.,offset[2]-delta_dist/4.])
    t1=time.time()
    while (time.time()-t1<0.01):
        continue 
    for a in range(rep):
        try:
            #Move to initial point
            gcs.MOV1([1,2,3],offset)
            t1=time.time()
            while (time.time()-t1<dwell_time):
                continue
            tnow=time.time()
            #move to point between in purpose of trigger mode and stay for 10ms
            gcs.MOV1([1,2,3],[offset[0],offset[1]+half,offset[2]])
            t1=time.time()
            while (time.time()-t1<0.01):
                continue           
            for i in range(int(x_points/2)):
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<dwell_time):
                        continue                   
                    print(2*i,j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                    if j!=y_points-2:
                        
                        gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1)+half,offset[2]])
                        t1=time.time()
                        while (time.time()-t1<0.01):
                            continue 
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i+half,offset[1]+delta_dist*(y_points-1)+half,offset[2]])
                t1=time.time()
                while (time.time()-t1<0.01):
                    continue                             
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-1),offset[2]])
                t1=time.time()
                while (time.time()-t1<dwell_time):
                    continue  
                print((2*i+1),j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                tnow= time.time()
                
                
                for j in range(int(y_points)-1):
                    
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<dwell_time):
                        continue                  
                    print(2*i+1,y_points-j-2,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                    if j!=y_points-2:
                        
                        gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1)-half,offset[2]])
                        t1=time.time()
                        while (time.time()-t1<0.01):
                            continue
                if i==int(x_points/2)-1:
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i+half,offset[1]+delta_dist*(y_points-1)-half,offset[2]])
                    t1=time.time()
                    while (time.time()-t1<0.01):
                        continue
                        
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                    t1=time.time()
                    while (time.time()-t1<dwell_time):
                        continue  
                    print((2*i+1),j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                else:
                    gcs.MOV1([1,2,3],[offset[0]-half,offset[1]-half,offset[2]-half])
                    t1=time.time()
                    while (time.time()-t1<0.01):
                        continue 
        except KeyboardInterrupt:
            break

def point_by_point_snake_3(gcs,x_points,y_points,dwell_time,
                       delta_dist,offset,rep = 10 ,velocity=50):
    """
        V2 using trigger mode 2
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger on selected channel send signal when stage is on target
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
       velocity:
    """
    wait_time =dwell_time +delta_dist*1.0/velocity+0.2
    print(wait_time)
    gcs.VEL1([1,2,3],[velocity,velocity,velocity])
    start =time.time()
    gcs.CTO1(1,[2,3],[1,2])
    gcs.CTO1(2,[2,3],[2,2])
    gcs.CTO1(3,[2,3],[3,2])

    t1=time.time()
    print(0,0,t1-start)
    tnow= time.time()
    for a in range(rep):
        try:
            #Move to initial point
            gcs.MOV1([1,2,3],offset)
            t1=time.time()
            while (time.time()-t1<wait_time):
                continue
            tnow=time.time()     
            for i in range(int(x_points/2)):
                
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])

                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time()  
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1),offset[2]])
                t1=time.time()
                while (time.time()-t1<wait_time):
                    continue
                tnow=time.time() 
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time() 
                if i ==int(x_points/2)-1:
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time()
        except KeyboardInterrupt:
            break
                        
        